In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SequentialFeatureSelector as sfs

In [2]:
def SFS_Feature(indep_X,dep_Y):
    
        SFSlist1=[]
        SFSlist=[]
        
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        
        SFSmodellist=[RF]
        
        for i in   SFSmodellist:
            log_sfs = sfs(i, n_features_to_select=6 , tol=None, direction='forward', scoring=None, cv=None, n_jobs=None)
            log_fit = log_sfs.fit(indep_X, dep_Y)
            log_sfs_feature=log_fit.transform(indep_X)
            SFSlist1.append(log_sfs_feature)
            op=log_sfs.get_feature_names_out(input_features=None)
            SFSlist.append(op)
        return SFSlist1, SFSlist

def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test, sc
    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm

def random(X_train,y_train,X_test):
        
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    

def rfe_classification(accrf): 
    
    rfedataframe=pd.DataFrame(index=['Random'],columns=['Random'])

    for number,idex in enumerate(rfedataframe.index):
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe

In [3]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, dtype=int, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [4]:
SFSlist=SFS_Feature(indep_X,dep_Y)       

accrf=[]

In [5]:
SFSlist, SFSlist1=SFSlist

In [6]:
SFSlist

[array([[  3.        ,  57.48210526,  38.86890244,   1.        ,
           0.        ,   0.        ],
        [  2.        ,  22.        ,  34.        ,   1.        ,
           0.        ,   0.        ],
        [  1.        ,  23.        ,  34.        ,   0.        ,
           0.        ,   0.        ],
        ...,
        [  3.        , 115.        ,  26.        ,   1.        ,
           0.        ,   1.        ],
        [  0.        ,  80.        ,  38.86890244,   0.        ,
           0.        ,   1.        ],
        [  0.        ,  49.        ,  53.        ,   0.        ,
           0.        ,   0.        ]])]

In [7]:
SFSlist1

[array(['al', 'bu', 'pcv', 'sg_c', 'sg_d', 'dm_yes'], dtype=object)]

In [8]:
for i in SFSlist:   
    X_train, X_test, y_train, y_test, sc=split_scalar(i,dep_Y)   
      
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)
    
result=rfe_classification(accrf)

In [9]:
result
#5

,Random
Random,0.99


In [10]:
import pickle
scaler_file="scaler_x.pkl"
pickle.dump(sc,open(scaler_file,"wb"))
sc=pickle.load(open("scaler_x.pkl",'rb'))


In [11]:
import pickle
filename="SFS_Sklearn_Forward.sav"

pickle.dump(classifier,open(filename,'wb'))

In [12]:
loaded_module=pickle.load(open("SFS_Sklearn_Forward.sav",'rb'))

In [13]:
preinput=sc.transform([[3,57.48,38.86,1,0,0]])
preinput

array([[ 1.75526219, -0.05492745, -0.00788715,  1.91621371, -0.46416285,
        -0.73029674]])

In [14]:
result1=loaded_module.predict(preinput)
result1

array([1])